In [11]:
import pyspark
from pyspark import SparkContext


sc = SparkContext.getOrCreate() 


data = sc.textFile("data/calidad_aire_datos_meteo_mes.csv") 
header = data.first() 


rows = data.filter(lambda line: line != header) \
  .map(lambda line: line.split(";")) 


rows_madrid = rows.filter(lambda f: f[0] == "28")


rows_temp = rows_madrid.filter(lambda f: f[3] == "83") 


resultado_ej1 = rows_temp.filter(
    lambda f: any(f[k] == "V" for k in range(9, min(57, len(f)), 2))
).count() 

print("Nº de registros de temperatura con >=1 hora válida:", resultado_ej1)

Nº de registros de temperatura con >=1 hora válida: 224
